In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().resolve().parents[0]))  # parent of /notebooks
from paths import data_path, report_path, print_layout
print_layout()

In [3]:
# 🧭 Environment Setup — capstone (corrected import names)
import sys, subprocess, importlib, shutil

# (install_name, import_name)
packages = [
    ("pandas", "pandas"),
    ("numpy", "numpy"),
    ("matplotlib", "matplotlib"),
    ("seaborn", "seaborn"),
    ("scikit-learn", "sklearn"),   # <-- package vs import name
]

print(f"🔍 Python: {sys.version.split()[0]}")
print("📦 Checking packages...\n")

def try_import(name):
    try:
        mod = importlib.import_module(name)
        ver = getattr(mod, "__version__", "OK")
        print(f"✅ {name} import OK (version: {ver})")
        return True
    except ImportError:
        print(f"⚠️ {name} not found")
        return False

missing = [(inst, imp) for inst, imp in packages if not try_import(imp)]

if missing:
    print("\n🧩 Installing missing via pip:", ", ".join(inst for inst, _ in missing))
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", *[inst for inst, _ in missing]])
    except Exception as e:
        print("❌ pip install failed:", e)

# Re-check
still_missing = []
for inst, imp in missing:
    if not try_import(imp):
        still_missing.append((inst, imp))

if still_missing:
    print("\n🚧 Still missing:", ", ".join(f"{imp} (pkg {inst})" for inst, imp in still_missing))
    print("👉 If you see this, verify Kernel → Python (ds311).")
else:
    print("\n🎉 Environment is ready!")


🔍 Python: 3.11.11
📦 Checking packages...

✅ pandas import OK (version: 2.3.3)
✅ numpy import OK (version: 2.3.3)
✅ matplotlib import OK (version: 3.10.7)
✅ seaborn import OK (version: 0.13.2)
✅ sklearn import OK (version: 1.7.2)

🎉 Environment is ready!


In [2]:
import sys; print(sys.executable); print(sys.version)

/opt/anaconda3/envs/ds311/bin/python
3.11.11 | packaged by conda-forge | (main, Dec  5 2024, 08:47:03) [Clang 18.1.8 ]


In [4]:
import pandas as pd
from pathlib import Path

# Adjust the path if needed
csv_path = Path("../data/diabetes.csv") # e.g., project/data/diabetes.csv
df = pd.read_csv(csv_path)

df.head(), df.shape, df.columns.tolist()[:20]

(   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
 0            6      148             72             35        0  33.6   
 1            1       85             66             29        0  26.6   
 2            8      183             64              0        0  23.3   
 3            1       89             66             23       94  28.1   
 4            0      137             40             35      168  43.1   
 
    DiabetesPedigreeFunction  Age  Outcome  
 0                     0.627   50        1  
 1                     0.351   31        0  
 2                     0.672   32        1  
 3                     0.167   21        0  
 4                     2.288   33        1  ,
 (768, 9),
 ['Pregnancies',
  'Glucose',
  'BloodPressure',
  'SkinThickness',
  'Insulin',
  'BMI',
  'DiabetesPedigreeFunction',
  'Age',
  'Outcome'])

In [2]:
import os
os.getcwd()

'/Users/b-rad.j.neiman/CODE/diabetes-capstone/notebooks'

In [6]:
import sqlite3

# Create or connect to the local database
conn = sqlite3.connect("../data/diabetes.db") # stored alongside your CSV

In [7]:
df.to_sql("diabetes_patients", conn, if_exists="replace", index=False)

768

In [8]:
pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)

,name
0,diabetes_patients


In [9]:
pd.read_sql_query("SELECT * FROM diabetes_patients LIMIT 5;", conn)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [10]:
conn.commit()
conn.close()